In [2]:
import math

def reward_function(params):
    
    reward = 1 #base reward
    Min_SPEED = 0.7 #min_speed to maintain
    
    # Read input parameters
    # track details
    track_width = params['track_width']
    distance_from_center = params['distance_from_center']
    on_track = params['all_wheels_on_track']
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    is_offtrack = params['is_offtrack']
    
    #car details
    speed = params['speed']
    is_left_of_center = params['is_left_of_center']
    heading = params['heading']
    progress = params['progress']
    steps = params['steps']
    steering_angle = params['steering_angle']
    
    #x and y co-ordinates
    x = params['x']
    y = params['y']
    
    
    # Rewarding if car on track
    def road_wheels(reward, on_track):
        if on_track ==  False:
        	reward *= 1e-3
        else:
          reward *= 3
        return reward
    
    #Rewarding if car stays near center line and punishing if it's far away from center line   	 
    def center_vehicle(reward, track_width, distance_from_center):
        
        marker_1 = 0.1 * track_width
        distance_from_border = abs(0.5*track_width-distance_from_center)
        
        # Give higher reward if the car is closer to center line and vice versa
        if distance_from_center <= marker_1:
            reward += 4
        if distance_from_border > 0.1:
            reward += 2
        else:
            reward -= 2
        return reward

    # Penalize if car steers too much
    def steer_vehicle(reward, waypoints, closest_waypoints, heading):
    	next_point = waypoints[closest_waypoints[1]]
    	prev_point = waypoints[closest_waypoints[0]]
    
    	# Calculate the direction in radius, arctan2(dy, dx), the result is (-pi, pi) in radians
    	track_direction = math.atan2(next_point[1] - prev_point[1], next_point[0] - prev_point[0])
    	# Convert to degree
    	track_direction = math.degrees(track_direction)
    
    	# Calculate the difference between the track direction and the heading direction of the car
    	direction_diff = abs(track_direction - heading)
    	if direction_diff > 180:
    	    direction_diff = 360 - direction_diff
    
    	# Penalize the reward if the difference is too large
    	DIRECTION_THRESHOLD = 10.0
    	if direction_diff > DIRECTION_THRESHOLD:
    	    reward /= 2
    
    	return reward
    
    #Penalize if speeds too much in the corners
    def turning_lookup(reward, waypoints, closest_waypoints, heading):
        
        next_point = waypoints[min(len(waypoints)-1, closest_waypoints[1]+2)]
        prev_point = waypoints[closest_waypoints[0]+1]
        
        # Calculate the direction in radius, arctan2(dy, dx), the result is (-pi, pi) in radians
        track_direction = math.atan2(next_point[1] - prev_point[1], next_point[0] - prev_point[0])
        # Convert to degree
        track_direction = math.degrees(track_direction)
        
        # Calculate the difference between the track direction and the heading direction of the car
        direction_diff = abs(track_direction - heading)
        if direction_diff > 180:
            direction_diff = 360 - direction_diff
            
        # Penalize the reward if the difference is too large by looking 2 points ahead
        DIRECTION_THRESHOLD = 30.0
        if direction_diff > DIRECTION_THRESHOLD:
            reward -= 2

            if speed > 0.8:
                reward -= 2
        return reward
        
    #Reward if the car completes track successfully    
    def speed_progress(reward, speed, progress):
        if progress > 90:
            reward += 10
        if speed < Min_SPEED:
            reward = reward - 1
        return reward
        
    
    reward = road_wheels(reward, on_track)
    reward = center_vehicle(reward, track_width, distance_from_center)
    reward = steer_vehicle(reward, waypoints,closest_waypoints, heading)
    reward = turning_lookup(reward, waypoints,closest_waypoints, heading)
    reward = speed_progress(reward, speed, progress)
    
    return float(reward)

